In [1]:
import boto3

In [2]:
s3=boto3.client('s3')

In [3]:
s3.list_buckets()

{'ResponseMetadata': {'RequestId': '4YK9RXA85T6VGBEJ',
  'HostId': 'SOW2L3KrvYJ7BxsujrBPUkbKe4z/4ae5bmgoLxaUb23HRVptnozfw5cFL7zUjxE9eUph8H7L0Y8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'SOW2L3KrvYJ7BxsujrBPUkbKe4z/4ae5bmgoLxaUb23HRVptnozfw5cFL7zUjxE9eUph8H7L0Y8=',
   'x-amz-request-id': '4YK9RXA85T6VGBEJ',
   'date': 'Sat, 28 Dec 2024 17:41:40 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [],
 'Owner': {'DisplayName': 'boreddyprahaladhareddy',
  'ID': 'ab93d5351552cae394fbb87b443f96c87747da8fe4bd5dea236ea465aecaf001'}}

In [4]:
def create_bucket(bucket_name):
  s3.create_bucket(Bucket=bucket_name)
  print('Bucket is created')

In [10]:
bucket_name = 'sreenu-bucket-1234'


In [11]:
create_bucket(bucket_name)

Bucket is created


In [12]:
s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'ZJTVQXKVCZ0SDTJ5',
  'HostId': 'QjLc64BXElvII1UDGfdom/s5SmfvkT4iSCLNOQXd/wAOteMiHf+dxij23sQAEhdnxD8ZodpZ+0o=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'QjLc64BXElvII1UDGfdom/s5SmfvkT4iSCLNOQXd/wAOteMiHf+dxij23sQAEhdnxD8ZodpZ+0o=',
   'x-amz-request-id': 'ZJTVQXKVCZ0SDTJ5',
   'date': 'Sat, 28 Dec 2024 17:46:15 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'sreenu-bucket-1234',
   'CreationDate': datetime.datetime(2024, 12, 28, 17, 46, tzinfo=tzutc())}],
 'Owner': {'DisplayName': 'boreddyprahaladhareddy',
  'ID': 'ab93d5351552cae394fbb87b443f96c87747da8fe4bd5dea236ea465aecaf001'}}

In [13]:
import os
def upload_file(file_path,object_name=None):
  if object_name is None:
    object_name=os.path.basename(file_path)
  s3.upload_file(file_path,bucket_name,object_name)
  

In [14]:
upload_file('creds/Prahalad_ReddyKEY.pem')

In [16]:
def list_objects(bucket_name):
  response=s3.list_objects_v2(Bucket=bucket_name)
  for obj in response['Contents']:
    print(obj)


In [25]:
list_objects(bucket_name)

{'Key': 'Prahalad_ReddyKEY.pem', 'LastModified': datetime.datetime(2024, 12, 28, 17, 51, 37, tzinfo=tzutc()), 'ETag': '"cddc810234be2f62b46a2f6ec7cc4c20"', 'Size': 1700, 'StorageClass': 'STANDARD'}
{'Key': 's3_key/Prahalad_ReddyKEY.pem', 'LastModified': datetime.datetime(2024, 12, 28, 18, 20, 52, tzinfo=tzutc()), 'ETag': '"cddc810234be2f62b46a2f6ec7cc4c20"', 'Size': 1700, 'StorageClass': 'STANDARD'}
{'Key': 's3_key/pragalad_EC2', 'LastModified': datetime.datetime(2024, 12, 28, 18, 20, 51, tzinfo=tzutc()), 'ETag': '"1cf688be20e3772e30ef2b43ef75bb17"', 'Size': 1700, 'StorageClass': 'STANDARD'}


In [18]:
def download_file(object_name,file_path):
  if not os.path.exists(os.path.dirname(file_path)):
    os.makedirs(os.path.dirname(file_path))
  s3.download_file(bucket_name,object_name,file_path)
  

In [19]:
download_file('Prahalad_ReddyKEY.pem','data_download/Prahalad_ReddyKEY.pem')

In [20]:
for root,dirs,files in os.walk('data_download'):
  print(root,dirs,files)

data_download [] ['Prahalad_ReddyKEY.pem']


In [21]:
def upload_directory(directory_path,s3_prefix):
  for root,dirs,files in os.walk(directory_path):
    for file in files:
      file_path=os.path.join(root,file).replace('\\','/')
      real_path=os.path.relpath(file_path,directory_path)
      print(real_path)
      s3_key=os.path.join(s3_prefix,real_path).replace('\\','/')
      s3.upload_file(file_path,bucket_name,s3_key)


In [24]:
upload_directory('creds','s3_key')

pragalad_EC2
Prahalad_ReddyKEY.pem


In [27]:
local_path='s3_download'
s3_prefix='s3_key'

In [29]:
paginator = s3.get_paginator('list_objects_v2')
for result in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix):
    if 'Contents' in result:
        for key in result['Contents']:
            s3_key = key['Key']

            # Build the file path
            file_path = os.path.join(local_path, os.path.relpath(s3_key, s3_prefix))
            
            # Ensure parent directories exist
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            
            # Download the file
            s3.download_file(bucket_name, s3_key, file_path)


In [30]:
def delete_objects():
  response=s3.list_objects_v2(Bucket=bucket_name)
  if 'Contents' in response:
    
    for obj in response['Contents']:
      s3.delete_object(Bucket=bucket_name,Key=obj['Key'])

delete_objects()      

In [31]:
s3.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'SE649VT11CGXYGH5',
  'HostId': 'UUNMPa196+hcbuLK03bVg57VAS9p//r7fLoUAyc5bOm7yF4WJubG3g/HIPQv3PzsCWYdtw/xoCjcAlBYddaBXj2tWfm142rnC/0G++fUykw=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'UUNMPa196+hcbuLK03bVg57VAS9p//r7fLoUAyc5bOm7yF4WJubG3g/HIPQv3PzsCWYdtw/xoCjcAlBYddaBXj2tWfm142rnC/0G++fUykw=',
   'x-amz-request-id': 'SE649VT11CGXYGH5',
   'date': 'Sat, 28 Dec 2024 18:58:33 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}